In [ ]:
%%capture
!git clone https://github.com/nyu-mll/jiant.git
%cd jiant
!pip install -r requirements-no-torch.txt
!pip install --no-deps -e ./
!pip install networkx==1.11

In [ ]:
import jiant.proj.main.tokenize_and_cache as tokenize_and_cache
import jiant.proj.main.export_model as export_model
import jiant.proj.main.scripts.configurator as configurator
import jiant.proj.main.runscript as main_runscript
import jiant.shared.caching as caching
import jiant.utils.python.io as py_io
import jiant.utils.display as display
import os
import sys
from google.colab import drive
# Adjust File path/partitions
sys.path.insert(0, "/content/jiant")
drive.mount('/content/gdrive')

In [ ]:
%%capture
%cd /content
# Download the tasks to be used
!PYTHONPATH=/content/jiant python jiant/jiant/scripts/download_data/runscript.py \
    download \
    --tasks pawsx \
    --output_path=/content/tasks/

In [ ]:
export_model.export_model(
    hf_pretrained_model_name_or_path="bert-base-multilingual-cased",
    output_base_path="./models/bert-base-multilingual-cased",
)

Some weights of BertForPreTraining were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenize and cache each task
languages = ['en', 'de', 'es', 'fr', 'ja', 'ko', 'zh']
task_names = ["pawsx_"+lan for lan in languages]
for task_name in task_names:
    tokenize_and_cache.main(tokenize_and_cache.RunConfiguration(
        task_config_path=f"./tasks/configs/{task_name}_config.json",
        hf_pretrained_model_name_or_path="bert-base-multilingual-cased",
        output_dir=f"./cache/{task_name}",
        phases=["val"],
    ))

In [ ]:
# Tokenize and cache each task
languages = ["af","ar","bg","de","el","en","et","eu","fa","fi","fr","he","hi","hu","id","it","ja","kk","ko","mr","nl","pt","ru","ta",
             "te","th","tl","tr","ur","vi","yo","zh"]
#task_names = ["udpos_{}".format(lang) for lang in languages]
task_names = ["udpos_en"]
for task_name in task_names:
  try:
    tokenize_and_cache.main(tokenize_and_cache.RunConfiguration(
        task_config_path=f"./tasks/configs/{task_name}_config.json",
        hf_pretrained_model_name_or_path="bert-base-multilingual-cased",
        output_dir=f"./cache/{task_name}",
        phases=["train","val"],
    ))
  except:
    print(task_name)


UdposTask
  [train]: /content/tasks/data/udpos_en/train-en.tsv
  [val]: /content/tasks/data/udpos_en/val-en.tsv
  [test]: /content/tasks/data/udpos_en/test-en.tsv


In [ ]:
# Build the config. file needed for training with Jiant
jiant_run_config = configurator.SimpleAPIMultiTaskConfigurator(
    task_config_base_path="./tasks/configs",
    task_cache_base_path="./cache",
    val_task_name_list=task_names,
    train_batch_size=8,
    eval_batch_size=8,
    epochs=3,
    num_gpus=1,
).create_config()
os.makedirs("./run_configs/", exist_ok=True)
py_io.write_json(jiant_run_config, "./run_configs/jiant_run_config.json")
display.show_json(jiant_run_config)

In [ ]:
# Start Training/Evaluating the model on the chosen tasks
run_args = main_runscript.RunConfiguration(
    jiant_task_container_config_path="./run_configs/jiant_run_config.json",
    output_dir="./runs/run_udpos",
    hf_pretrained_model_name_or_path="bert-base-multilingual-cased",
    model_path="/content/gdrive/MyDrive/thesis/multi_copa-multirc-paws/run1/best_model.p",
    model_config_path="./models/bert-base-multilingual-cased/model/config.json",
    learning_rate=2e-5,
    eval_every_steps=500,
    
    do_train=False,
    do_val=True,
    do_save = True,
    force_overwrite=False
)

main_runscript.run_loop(run_args)

In [ ]:
run_args = main_runscript.RunConfiguration(
    jiant_task_container_config_path="./run_configs/jiant_run_config.json",
    output_dir="./runs/run1",
    hf_pretrained_model_name_or_path="bert-base-multilingual-cased",
    model_path="./models/bert-base-multilingual-cased/model/model.p",
    model_config_path="./models/bert-base-multilingual-cased/model/config.json",
    learning_rate=2e-5,
    eval_every_steps=500,
    
    do_train=True,
    do_val=True,
    do_save = True,
    force_overwrite=True,
)

main_runscript.run_loop(run_args)

In [ ]:
!cp -r '/content/runs' '/content/gdrive/MyDrive/thesis/udpos_multitrained'

In [ ]:
!cp -r '/content/run_configs' '/content/gdrive/MyDrive/thesis/multi-trained/config'